In [ ]:
import json

import requests
from env import SECRET, KEY #secret sauce


In [ ]:
# FLICKR API
# flickr.photos.search

URL = 'https://api.flickr.com/services/rest/?method=flickr.photos.search'

# Param
# written as called

# general
per_page = '500'
page = '1'
format_output = 'json' # call exclude _output
extras = "description, license, date_upload, date_taken, owner_name, icon_server, original_format, last_update, geo, tags, url_sq"
nojsoncallback = '1'
has_geo = '1'

# search specific

# core
lat = '47'
lon = '7'
radius = '2'
radius_units = 'km'

# periphery
tags = ''
min_taken_date = '' # unix
max_taken_date = '' # unix
accuracy = ''





In [95]:
def executeSearch(page= 1):

    params = dict(api_key= KEY, per_page= per_page, page= page, format= format_output, extras= extras, nojsoncallback= nojsoncallback,
        has_geo= has_geo, accuracy= accuracy, lat= lat, lon= lon, radius= radius)
    
    r = requests.get(url= URL, params= params)
    

    print(f'Status: {r}')
    # print(json.dumps(r.json(), indent=2))

    # file must be threadsafe
    with open(f'search_USER_PART', 'w') as f:
        json.dump(r_json, f)
        f.close()

    # handled by either db or sessions in production
    total_page = response['photos']['pages']
    current_page = response['photos']['page']


    
    
    

    


In [97]:
executeSearch()

with open(f'/response/search_USER_PART', 'r') as f:
        response = json.load(f)

try:
    total_photos = response['photos']['total']
    current_page = response['photos']['page']
    per_page = response['photos']['perpage']
    total_page = response['photos']['pages']

    print(f'Page {current_page} of {total_page}')

except:
    print("unexpected return format")


while current_page < total_page:
    executeSearch(page= current_page)
    print(f'Page {current_page} of {total_page}')
    current_page += 1
    





Status: <Response [200]>
Page 1 of 5
Status: <Response [200]>
Page 1 of 5
Status: <Response [200]>
Page 2 of 5
Status: <Response [200]>
Page 3 of 5


KeyboardInterrupt: 

trim

{"photos": {"page": 1, "pages": 5, "perpage": 250, "total": "1141",

"photo": [{"id": "50334344962", "owner": "59453656@N02", "secret": "90a12208fb", "server": "65535", "farm": 66, "title": "2020, Neuch\u00e2tel - Ikonta 521 & Fomapan 400 in D76", "ispublic": 1, "isfriend": 0, "isfamily": 0, "license": "0", "description": {"_content": ""}, "dateupload": "1599929398", "lastupdate": "1600111295", "datetaken": "2020-09-12 18:49:51", "datetakengranularity": 0, "datetakenunknown": "1", "ownername": "FourthRockFromTheSun", "iconserver": "2745", "iconfarm": 3, "tags": "fomapan ikonta zeiss 521 neuch\u00e2tel", "originalsecret": "d6dd695d87", "originalformat": "jpg", "latitude": "47.006655", "longitude": "6.971404", "accuracy": "16", "context": 0, "place_id": "", "woeid": "6692332", "geo_is_public": 1, "geo_is_contact": 0, "geo_is_friend": 0, "geo_is_family": 0, "url_sq": "https://live.staticflickr.com/65535/50334344962_90a12208fb_s.jpg", "height_sq": 75, "width_sq": 75},
...
}

In [ ]:

# with open('example_response.json', 'w') as f:
#     json.dump(r_json, f)